In [1]:
from mlagents.torch_utils.torch import torch
import numpy as np

from mlagents.plugins.bvh_utils.visualize import skeletons_plot, motion_animation
from mlagents.plugins.dataset.dataset import SkeletonInfo
import mlagents.plugins.bvh_utils.lafan_utils as utils
from mlagents.plugins.dataset.skeleton_side_channel import Skeleton_SideChannel 

import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib qt

from mlagents_envs.environment import UnityEnvironment

# INITIALIZATION OF UNITY SAVED DATA AND UNITY ENV

In [2]:
motion_path = "../ml-agents/mlagents/plugins/skeleton_aware_op/data/LaFan/Real_Data/"
# motion_path = "../../LaFan_BVH/rotation_test/"
skdata = SkeletonInfo(skdata_path = './data_legs/skdata.txt')

In [3]:
# Setup Unity Environment + sidechannel
skeleton_sidechannel = Skeleton_SideChannel()

try:
    env.close()
except:
    pass

# filename = None enables to communicate directly with the unity editor
env = UnityEnvironment(file_name=None, seed=1, side_channels=[skeleton_sidechannel])
env.reset()

# DATA RETRIVAL FROM FILE AND ENV

In [25]:
with open('./data_legs/walk1_subject1_legs_csv.txt') as f:
    lines = f.readlines()

n_lines = len(lines)
n_features = len(lines[0].split(','))

features = torch.zeros((n_lines,n_features)).float()

for i, line in enumerate(lines):
    line = line
    line_array = [float(f) for f in line.split(',')]
    features[i] = torch.tensor(line_array).float()


In [18]:
with open('./data_legs/skdata.txt') as f:
    lines = f.readlines()

# EXTRACT INITIAL INFO ABOUT SKELETON
n_joints = int(lines[0][:-1])
init_positions = torch.tensor( [float(f) for f in lines[1][:-1].split(',')] ).float().reshape(n_joints, 3)
init_rotations =  torch.tensor( [float(f) for f in lines[2][:-1].split(',')] ).float().reshape(n_joints, 4)
parents = [int(f) for f in lines[3][:-1].split(',')]
frametime = float(lines[4][:-1])

In [6]:
behavior_name = list(env.behavior_specs)[0]
spec = env.behavior_specs[behavior_name]
env.reset()

# get action and obs dimension
state_dim = spec.observation_specs[0].shape[0]
action_dim = spec.action_spec.continuous_size

decision_steps, terminal_steps = env.get_steps(behavior_name)

unity_data = []

for i in range(60):
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    unity_data.append(decision_steps.obs[0])
    
    # Generate an action for all agents
    action = spec.action_spec.empty_action(len(decision_steps))

    # Set the actions
    env.set_actions(behavior_name, action)

    # Move the simulation forward
    env.step()

# DATA PROCESSING

In [26]:
features  = features.reshape(features.shape[0],skdata.num_joints,10)

csv_positions = features[:,:,:3]
csv_rotations = features[:,:,3:7]
csv_velocity = features[:,:,7:]
csv_local_positions = csv_positions[:] - csv_positions[:,0:1,:]

In [34]:
env_velocity[0:20,0]

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.7680, -0.6159,  0.4227],
        [ 0.6556, -0.9629,  0.2581],
        [ 0.5239, -1.0636,  0.3037],
        [ 0.4717, -0.9220,  0.2196],
        [ 0.5710, -0.0405,  0.4119],
        [ 0.6274,  0.4310,  0.3977],
        [ 0.6495,  0.4341,  0.3041],
        [ 0.6799,  0.2477,  0.2422],
        [ 0.7265,  0.0542,  0.1995],
        [ 0.7712, -0.1483,  0.1213],
        [ 0.8151, -0.3389,  0.0243],
        [ 0.8701, -0.5066,  0.0640],
        [ 0.9258, -0.7176,  0.3259],
        [ 1.0055, -0.8989,  0.6287],
        [ 1.1362, -1.0300,  0.8506],
        [ 1.2830, -1.1897,  0.8983],
        [ 1.4184, -1.4777,  0.8835],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.4268, -0.6792,  1.2210]])

In [8]:
# EXTRACT INITIAL INFO ABOUT SKELETON
n_joints = int(skeleton_sidechannel.msg[0])
init_rotations = torch.tensor(skeleton_sidechannel.msg[1:1+n_joints*4]).float().reshape(n_joints,4)
init_positions = torch.tensor(skeleton_sidechannel.msg[ 1+n_joints*4 : 1+n_joints*4 + n_joints*3 ]).float().reshape(n_joints,3)
parents = [int(f) for f in skeleton_sidechannel.msg[ 1+n_joints*4 + n_joints*3 : -1 ]]
frametime = skeleton_sidechannel.msg[ -1 ]

AttributeError: 'Skeleton_SideChannel' object has no attribute 'msg'

In [9]:
n = 54

features_per_joint = 13

obs = torch.cat([torch.tensor(unity_data[f]).float() for f in range(60)],dim=0).float()
print(obs.shape)

# extract only the observation corresponding to the joints
joint_features = obs[0:n,:-6]
print(joint_features.shape)
joint_features = joint_features.reshape(n,-1, 13)


# extract information from the observation vector
env_velocity = joint_features[:,:,:3].clone()
env_angular_vel = joint_features[:,:,3:6].clone()
env_positions = joint_features[:,:,6:9].clone()
env_rotations = joint_features[:,:,9:].clone()

# unity rotation is x,y,z,w and lafan is w,x,y,z, change to the latter
temp = env_rotations[:,:,3].clone()
env_rotations[:,:,1:] = env_rotations[:,:,:3] 
env_rotations[:,:,0] = temp

env_local_positions = env_positions[:] - env_positions[:,0:1,:]

torch.Size([57, 162])
torch.Size([54, 156])


In [27]:
# extract velocity
env_root_pos, env_glob_pos = utils.get_global_position_from_velocity(torch.tensor([0,0,0]).float(), env_velocity[:,0,:], 1, env_positions)
csv_root_pos, csv_pos_from_vel = utils.get_global_position_from_velocity(torch.tensor([0,0,0]).float(), csv_velocity[:,0,:], 10, csv_positions)

In [29]:
# reconstruct pose from rotations
offsets = init_positions.clone()
offsets = offsets.reshape(1,skdata.num_joints,3)
offsets = offsets.repeat(env_rotations.shape[0],1,1)

# env_rotations[:,0,:] = torch.tensor([1.,0.,0.,0.]).float()
# csv_rotations[:,0,:] = torch.tensor([1.,0.,0.,0.]).float()

_, env_pos_from_rot = utils.quat_fk(env_rotations, offsets, skdata.parents)

# reconstruct pose from rotations
offsets = init_positions.clone()
offsets = offsets.reshape(1,skdata.num_joints,3)
offsets = offsets.repeat(csv_rotations.shape[0],1,1)

_, csv_pos_from_rot = utils.quat_fk(csv_rotations[:,:,:], offsets, skdata.parents)


In [12]:
# reconstruct the posiitons from velocity
csv_pos_from_vel = torch.zeros_like(csv_velocity)
csv_prev_pos = csv_local_positions[0]
env_pos_from_vel = torch.zeros_like(env_velocity)
env_prev_pos = env_local_positions[0]

for i in range(1,csv_velocity.shape[0]):
    csv_pos_from_vel[i-1,:] = csv_prev_pos
    csv_prev_pos = csv_prev_pos + csv_velocity[i,:] * 0.05

for i in range(1, env_velocity.shape[0]):
    env_pos_from_vel[i-1,:] = env_prev_pos
    env_prev_pos = env_prev_pos + env_velocity[i,:] * 0.05
    if torch.abs(env_velocity[i].sum())<0.00001 :
        env_prev_pos = env_local_positions[i]

# VISUALIZE

In [22]:
# POSE FROM POSITIONS
limits = [[-1,1],[-1,1],[-1,1]]
fig, ax = skeletons_plot([csv_local_positions[100].cpu().detach(), env_local_positions[20].cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits=limits, return_plot=True)
# fig,ax=skeletons_plot([csv_local_positions[100].cpu().detach()], [skdata.edges], ['g'], limits=limits, return_plot=True)

plt.show()

In [23]:
# POSE FROM ROTATIONS
limits = [[-1,1],[-1,1],[-1,1]]
fig,ax=skeletons_plot([csv_pos_from_rot[0].cpu().detach(), env_pos_from_rot[25].cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits=limits, return_plot=True)
plt.show()

In [30]:
# MOTION FROM POSITIONS
limits = [[-1,1],[-1,1],[-1,1]]
# anim = motion_animation([csv_local_positions[:55].cpu().detach(), env_local_positions.cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits)
# anim = motion_animation([csv_local_positions[100:270].cpu().detach(), csv_pos_from_vel[100:270].cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits)
# anim = motion_animation([env_local_positions.cpu().detach(), env_pos_from_vel.cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits)
# anim = motion_animation([env_local_positions.cpu().detach(), env_pos_from_rot.cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits)
anim = motion_animation([csv_local_positions[100:270].cpu().detach(), csv_pos_from_rot[100:270].cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits)
# anim = motion_animation([csv_pos_from_vel[0:100].cpu().detach()], [skdata.edges], ['g'], limits)
# HTML(anim.to_jshtml())

In [16]:
env.close()